In [1]:
%load_ext autoreload
%autoreload 2


# Load relevent data

rosters

processed possession data

In [2]:
from nba import NbaTracker
from nba_api.stats.static.players import find_player_by_id
from nba_api.stats.static.teams import find_team_name_by_id

nbaTracker = NbaTracker()
season_name = "2015-16"
nbaTracker.add_season(season_name)
season = nbaTracker.seasons[season_name]
season.load_possession_data()
season.build_player_seasons()
games = season.games


also get player tracking data

Create train, test sets

In [21]:
from nba_dataclasses import EventType, ResultClass 
import pandas as pd 


rebounds = []
columns = ['result_class', 'result_team', 'num_fts', 'shot_type', 'is_blocked', 'is_putback', 'in_penalty', 'offense_is_home', 'score_margin']
player_attributes = []
# result_classes = []
rb_count = 0
for team in ['off', 'def']:
    for player_num in [0, 1, 2, 3, 4]:
        player = 'player' + team + str(player_num)
        player_attributes.append(player)
        columns.append(player)
for game_id, game in games.items():
    last_game_event = None
    for ge in game.game_events:
        if ge.event_type == EventType.Rebound:
            rb_count += 1
            result_class = ge.result.result_class
            if result_class == ResultClass.JUMPBALL:
                continue
                result_team = 2
            elif result_class in {
                ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}:
                result_team = 1
            else: 
                result_team = 0
            num_fts = ge.result.num_fts
            shot_type = ge.result.shot_type
            is_blocked = ge.result.is_blocked
            is_putback = last_game_event.is_putback
            in_penalty = ge.in_penalty
            offense_is_home = ge.offense_is_home
            score_margin = ge.score_margin
            rebound = [
                result_class,
                result_team,
                num_fts,
                shot_type,
                is_blocked,
                is_putback,
                in_penalty,
                offense_is_home,
                score_margin,
            ]
            for team in [0, 1]:
                for player_num in [0, 1, 2, 3, 4]:
                    rebound.append(ge.lineup.lineup[team][player_num])
            
            # result_classes.append(result_class)
            rebounds.append(rebound)
        last_game_event = ge

rebounds = pd.DataFrame(rebounds, columns=columns)
   


In [22]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=43)
for train_index, test_index in split.split(rebounds, rebounds['result_team']):
    train = rebounds.loc[train_index]
    test = rebounds.loc[test_index]


In [23]:
train_X = train.drop(['result_class', 'result_team', 'num_fts'], axis=1)
train_y = train['result_team']
test_X = test.drop(['result_class', 'result_team', 'num_fts'], axis=1)
test_y = test['result_team']



# discover rebound data

In [24]:
train.groupby('num_fts')['result_team'].value_counts()

num_fts  result_team
0        0              69963
         1              26028
2        0                264
         1                262
Name: result_team, dtype: int64

In [8]:
test_y.shape

(24181,)

# Prepare data

build add player data transformer

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from nba_dataclasses import ResultClass
from nba_dataclasses import PlayerInfo
import numpy as np
from sklearn import linear_model
from build_shot_chance_data import ShotType

shot_type_index = 0
is_blocked_index = 1
lineup_start_index = 6

## TODO generate rebound stats and rebound foul rate by player/lineup for off/def and add to PlayerInfo
## add those stats to X in transform

class AddRebPlusMinus(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass 

    def fit(self, X, y):
        """Generate adjusted rebound plus/minus from data.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            rebound data.
        y : {array-like, sparse matrix}, shape (n_samples, )
            result_class of rebound.
            
        Returns
        -------
        self : object
            Returns self.
        """
        player_index = 0
        player_indicies: dict[int, PlayerInfo] = {}
        for player, ps in nbaTracker.seasons[season_name].player_seasons.items():
            if player not in player_indicies:
                player_indicies[player] = PlayerInfo(player_index)
                player_index += 1
            for team in ps.player_contributions:
                player_indicies[player].teams.add(team)
        num_players = player_index


        reb_by_shottype = {st: {is_blkd: {True: 0, False: 0}
                                for is_blkd in [False, True]} for st in ShotType}
        lineup_index = 0
        lineup_indicies: dict[tuple[int, int, int, int,
                                  int, int, int, int, int, int], int] = {}

        for rebound, result_team in zip(X, y):
            if result_team == 2: #EventType.JumpBall:
                continue
            is_oreb = result_team == 1 # in {
               # ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
            shot_type = rebound[shot_type_index]
            is_blocked = rebound[is_blocked_index]

            reb_by_shottype[ShotType(shot_type)][is_blocked][is_oreb] += 1

            lineup = tuple(rebound[lineup_start_index:lineup_start_index+10])
            if lineup not in lineup_indicies:
                lineup_indicies[lineup] = lineup_index
                lineup_index += 1
        num_lineups = lineup_index

        for st in ShotType:
            for is_blkd in [False, True]:
                total = reb_by_shottype[st][is_blkd][False] + \
                    reb_by_shottype[st][is_blkd][True]
                if total == 0:
                    continue
                reb_by_shottype[st][is_blkd][False] = reb_by_shottype[st][is_blkd][False] / total
                reb_by_shottype[st][is_blkd][True] = reb_by_shottype[st][is_blkd][True] / total
        
        oreb_chance_dif = np.zeros(num_lineups)
        lineups = np.zeros((num_lineups, num_players*2))
        sample_weights = np.zeros(num_lineups)

        for rebound, result_team in zip(X, y):
            if result_team == 2: # EventType.JumpBall:
                continue
            is_oreb = result_team == 1 # in {
              #  ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
            shot_type = rebound[shot_type_index]
            is_blocked = rebound[is_blocked_index]
            oreb_chance = reb_by_shottype[ShotType(shot_type)][is_blocked][True]

            lineup = tuple(rebound[lineup_start_index:lineup_start_index+10])
            oreb_chance_dif[lineup_indicies[lineup]
                            ] += (1 if is_oreb else 0) - oreb_chance
            sample_weights[lineup_indicies[lineup]] += 1
        oreb_chance_dif /= sample_weights
        
        for lineup, lineup_index in lineup_indicies.items():
            for team_index in [0, 1]:
                for player_index in [0, 1, 2, 3, 4]:
                    player_id = lineup[team_index*5 + player_index]
                    i = player_indicies[player_id].index + team_index*num_players
                    lineups[lineup_index, i] = 1

        reg = linear_model.Ridge(1000)
        
        reg.fit(lineups, oreb_chance_dif, sample_weights)

        for player in player_indicies.values():
            player.oreb_pm = reg.coef_[player.index]
            player.dreb_pm = reg.coef_[player.index + num_players]

        max_count = 5
        count = 0
        print("Top", max_count, "REB_PM")
        for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=True):
            name = find_player_by_id(pid)['full_name']
            print(name, player.oreb_pm, player.dreb_pm)
            count += 1
            if count == max_count:
                break
        count = 0

        print()
        print("Bottom", max_count, "REB_PM")
        for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=False):
            name = find_player_by_id(pid)['full_name']
            print(name, player.oreb_pm, player.dreb_pm)
            count += 1
            if count == max_count:
                break

        self.player_indicies = player_indicies

        return self 

    def transform(self, X, y=None):
        pms = np.zeros(len(X))
        for i, rebound in enumerate(X):
            for pid in rebound[lineup_start_index:lineup_start_index+5]:
                pms[i] += self.player_indicies[pid].oreb_pm
            for pid in rebound[lineup_start_index+5:lineup_start_index+10]:
                pms[i] += self.player_indicies[pid].dreb_pm
        return np.c_[pms]

build data preparing pipelines

In [31]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, OrdinalEncoder
from sklearn.pipeline import make_pipeline

# transform catagorical attributes to one-hot encoding
catagorical_attributes = [0] # ['shot_type']
pass_attributes = [1,2,3] #['is_blocked', 'is_putback', 'in_penalty']
add_pm_attributes = list(range(14)) # columns[2:]
# oe = OrdinalEncoder()
# oe.fit(train_y.to_numpy().reshape(-1, 1))

preprocess = ColumnTransformer([("cat", OneHotEncoder(
), catagorical_attributes), ("pass", 'passthrough', pass_attributes)])

preprocess_w_pm = ColumnTransformer([("cat", OneHotEncoder(
), catagorical_attributes), ("pass", 'passthrough', pass_attributes), ("add_pm", AddRebPlusMinus(), add_pm_attributes)])

# add_pm_pipe = make_pipeline(AddRebPlusMinus(), preprocess_w_pm)


In [27]:
train_y.shape, test_y.shape

((96517,), (24130,))

In [188]:
# processed_train_y = oe.transform(train_y.to_numpy().reshape(-1,1))
# processed_test_y = oe.transform(test_y.to_numpy().reshape(-1,1))


In [32]:
processed_train_X = preprocess.fit_transform(train_X)
processed_test_X = preprocess.transform(test_X)

In [33]:
pm_train_X = preprocess_w_pm.fit_transform(
    train_X.to_numpy(dtype=int), train_y.to_numpy(dtype=int))
pm_test_X = preprocess_w_pm.transform(
    test_X.to_numpy(dtype=int))


Top 5 REB_PM
Joakim Noah 0.022651136422264843 -0.010021228254741105
Enes Kanter 0.03125599306395365 -0.0010771641627129683
Nikola Jokic 0.01282026664499899 -0.01885135646486912
Zach Randolph 0.026998035604780642 -0.0028838457360115173
Kenneth Faried 0.0176411554074841 -0.01204769452658252

Bottom 5 REB_PM
Hollis Thompson -0.008086491879452012 0.022984507153118987
Chandler Parsons -0.022883955347664908 0.00816004699600234
Jerami Grant -0.020071559172109305 0.007586548961598298
Kendall Marshall -0.013913554798802926 0.013065011812392342
Roy Hibbert -0.01985457838692857 0.006562738961239133


In [42]:
pm_train_X[:5]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        , -0.00107375],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.00395573],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.00872206],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.07200159],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.01128726]])

# Build averages model to compare to

In [45]:
import tensorflow as tf 
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [218]:
# oe.inverse_transform([[6]])

array([[10]], dtype=int64)

In [43]:
processed_train_X.shape, train_y.shape

((96517, 9), (96517,))

In [41]:
train_y.to_numpy()[:5], processed_train_X[:5]


(array([1, 1, 0, 0, 0], dtype=int64),
 array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 1., 0.]]))

In [47]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True) #tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

# fit data
model.fit(processed_train_X, train_y, epochs=3)


Epoch 1/3
3017/3017 [==============================] - 41s 13ms/step - loss: 0.5700 - accuracy: 0.7273
Epoch 2/3
3017/3017 [==============================] - 40s 13ms/step - loss: 0.5694 - accuracy: 0.7272
Epoch 3/3
3017/3017 [==============================] - 40s 13ms/step - loss: 0.5694 - accuracy: 0.7265


In [48]:
model.evaluate(processed_test_X, test_y, verbose=2)


755/755 - 5s - loss: 0.5691 - accuracy: 0.7244 - 5s/epoch - 7ms/step


[0.5690756440162659, 0.7243680357933044]

# Build rebound chance model with player data

In [49]:
import tensorflow as tf
mean, variance = pm_train_X.mean(), pm_train_X.var()
pm_model = tf.keras.models.Sequential([
    # tf.keras.layers.Normalization(mean=mean, variance=variance),
    # tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
pm_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

# fit data
pm_model.fit(pm_train_X, train_y, epochs=3)


Epoch 1/3
3017/3017 [==============================] - 35s 11ms/step - loss: 0.5832 - accuracy: 0.7111
Epoch 2/3
3017/3017 [==============================] - 33s 11ms/step - loss: 0.5665 - accuracy: 0.7269
Epoch 3/3
3017/3017 [==============================] - 33s 11ms/step - loss: 0.5652 - accuracy: 0.7268


In [50]:
pm_model.evaluate(pm_test_X, test_y, verbose=2)


755/755 - 6s - loss: 0.5663 - accuracy: 0.7266 - 6s/epoch - 7ms/step


[0.5662544369697571, 0.7266058921813965]

compare predictions

In [229]:
predictions = model(processed_train_X[:5]).numpy()

predictions2 = pm_model(pm_train_X[:5]).numpy()
print(tf.nn.softmax(predictions).numpy())
print(tf.nn.softmax(predictions2).numpy())

[[0.00134039 0.05730633 0.13375078 0.00380812 0.00187581 0.21288466
  0.5890339 ]
 [0.00395427 0.0581353  0.0581065  0.00215441 0.00379258 0.16888256
  0.7049744 ]
 [0.00505907 0.06223856 0.10497802 0.0007671  0.00322577 0.18744768
  0.63628376]
 [0.00377648 0.04132847 0.03728819 0.00388282 0.00296327 0.23558995
  0.67517084]
 [0.00505907 0.06223856 0.10497802 0.0007671  0.00322577 0.18744768
  0.63628376]]
[[2.02489481e-03 5.85501306e-02 1.43772364e-01 4.16370900e-03
  1.84199691e-03 2.28293180e-01 5.61353743e-01]
 [3.91879957e-03 5.41324876e-02 5.02987579e-02 1.94767967e-03
  3.62257496e-03 1.45336151e-01 7.40743518e-01]
 [3.58494231e-03 6.26809224e-02 1.08168766e-01 7.41173921e-04
  3.25409416e-03 2.16465607e-01 6.05104506e-01]
 [3.57386866e-03 3.74858864e-02 3.45067866e-02 4.39155288e-03
  3.44617898e-03 2.13142738e-01 7.03453064e-01]
 [3.48156109e-03 6.07872009e-02 1.02972575e-01 7.29075749e-04
  3.18953837e-03 1.98538855e-01 6.30301237e-01]]


In [231]:
processed_train_y[:5]


array([[5.],
       [6.],
       [6.],
       [6.],
       [6.]])

# Build Rebound Chance APM

build rebound chance by shottype

In [11]:
from nba_dataclasses import ResultClass
from build_shot_chance_data import ShotType

reb_by_shottype = {st: {is_blkd: {True: 0, False: 0}
                        for is_blkd in [False, True]} for st in ShotType}
# reb_by_blkd = {is_blkd: {True: 0, False: 0} for is_blkd in [False, True]}
total_rebs = len(rebounds)

for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    is_oreb = rebound.result.result_class in {
        ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
    reb_by_shottype[ShotType(rebound.result.shot_type)
                    ][rebound.result.is_blocked][is_oreb] += 1
    # reb_by_blkd[rebound.result.is_blocked][is_oreb] += 1

for st in ShotType:
    for is_blkd in [False, True]:
        total = reb_by_shottype[st][is_blkd][False] + \
            reb_by_shottype[st][is_blkd][True]
        if total == 0:
            continue
        reb_by_shottype[st][is_blkd][False] = reb_by_shottype[st][is_blkd][False] / total
        reb_by_shottype[st][is_blkd][True] = reb_by_shottype[st][is_blkd][True] / total

reb_by_shottype


{<ShotType.AtRim: 1>: {False: {True: 0.36029926260832124,
   False: 0.6397007373916788},
  True: {True: 0.4172010562052056, False: 0.5827989437947945}},
 <ShotType.ShortMidRange: 2>: {False: {True: 0.28560246563279684,
   False: 0.7143975343672032},
  True: {True: 0.41282375236891977, False: 0.5871762476310802}},
 <ShotType.LongMidRange: 3>: {False: {True: 0.20750070647127689,
   False: 0.7924992935287231},
  True: {True: 0.34275618374558303, False: 0.657243816254417}},
 <ShotType.Arc3: 4>: {False: {True: 0.24870375560538116,
   False: 0.7512962443946188},
  True: {True: 0.41015625, False: 0.58984375}},
 <ShotType.Corner3: 5>: {False: {True: 0.24753747323340472,
   False: 0.7524625267665953},
  True: {True: 0.4918032786885246, False: 0.5081967213114754}},
 <ShotType.FreeThrowNR: 6>: {False: {True: 0, False: 0},
  True: {True: 0, False: 0}},
 <ShotType.FreeThrowR: 7>: {False: {True: 0.115212042263714,
   False: 0.884787957736286},
  True: {True: 0, False: 0}}}

build and solve reb apm

In [12]:
# y = is offenseve rebound chance - chance from shot type
# X = player lineups
# solve for player rebound +/-

from nba_dataclasses import PlayerInfo
import numpy as np
from sklearn import linear_model

player_index = 0
player_indicies: dict[int, PlayerInfo] = {}
lineup_index = 0
lineup_indicies: dict[tuple[tuple[int, int, int, int,
                                  int], tuple[int, int, int, int, int]], int] = {}

for player, ps in nbaTracker.seasons[season_name].player_seasons.items():
    if player not in player_indicies:
        player_indicies[player] = PlayerInfo(player_index)
        player_index += 1
    for team in ps.player_contributions:
        player_indicies[player].teams.add(team)
for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    lineup = rebound.lineup.lineup
    if lineup not in lineup_indicies:
        lineup_indicies[lineup] = lineup_index
        lineup_index += 1
num_lineups = lineup_index
num_players = player_index

y = np.zeros(num_lineups)
ey = np.zeros(num_lineups)
X = np.zeros((num_lineups, num_players*2))
sample_weights = np.zeros(num_lineups)

for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    lineup = rebound.lineup.lineup
    is_oreb = rebound.result.result_class in {
        ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
    oreb_chance = reb_by_shottype[ShotType(rebound.result.shot_type)
                                  ][rebound.result.is_blocked][True]
    y[lineup_indicies[lineup]] += (1 if is_oreb else 0) - oreb_chance
    ey[lineup_indicies[lineup]] += oreb_chance
    sample_weights[lineup_indicies[lineup]] += 1

for lineup, lineup_index in lineup_indicies.items():
    for team_index in [0, 1]:
        for player_index in [0, 1, 2, 3, 4]:
            player_id = lineup[team_index][player_index]
            i = player_indicies[player_id].index + team_index*num_players
            X[lineup_index, i] = 1

reg = linear_model.BayesianRidge()
reg.fit(X, y, sample_weights)

for player in player_indicies.values():
    player.oreb_pm = reg.coef_[player.index]
    player.dreb_pm = reg.coef_[player.index + num_players]


Display best and worst rebounders

In [13]:
max_count = 20
count = 0
print("Top", max_count, "REB_PM")
for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=True):
    name = find_player_by_id(pid)['full_name']
    print(name, player.oreb_pm, player.dreb_pm)
    count += 1
    if count == max_count:
        break
count = 0

print()
print("Bottom", max_count, "REB_PM")
for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=False):
    name = find_player_by_id(pid)['full_name']
    print(name, player.oreb_pm, player.dreb_pm)
    count += 1
    if count == max_count:
        break


Top 20 REB_PM
Briante Weber 0.4492725284760221 0.060335966133836724
Steven Adams 0.2151893826964091 -0.15052424581813706
Zach Randolph 0.3010836669442734 -0.036267832456417015
Ryan Hollins 0.20950129841306978 -0.11502712870745921
Cristiano Felicio 0.18021016452830613 -0.13504201631584553
Noah Vonleh 0.1673449199841731 -0.1413508603893439
Dwight Howard 0.1879678500511335 -0.10824087801966858
Clint Capela 0.23542551621242028 -0.060258789922858644
Robin Lopez 0.1727087205849283 -0.11751434755578145
Willie Reed 0.1850612810552634 -0.10104373536817868
Lavoy Allen 0.1668373095911683 -0.11894373761894743
Andre Roberson 0.19226429183115162 -0.09252962911154458
Joakim Noah 0.2051026341889192 -0.07874715213897052
Nikola Jokic 0.1378810027206907 -0.14531160862936932
Alex Len 0.16697756302821215 -0.1033118190049501
Tim Duncan 0.10290011084190467 -0.16607825373438534
Jarrett Jack 0.19803616112030487 -0.07027270797498851
Rondae Hollis-Jefferson 0.16490752522474525 -0.09970997424281151
Omer Asik 0.00